# Setup and imports

In [24]:
import os
import pandas as pd
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
import torch
from torchvision import models, transforms
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

from src.utils import download_images
from src.feature_extractor import extract_features
from src.model_trainer import train_model

import warnings
warnings.filterwarnings('ignore')

# Helper Functions 

## Feature Extraction 

In [4]:
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

def preprocess_for_resnet(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        img_tensor = transform(img).unsqueeze(0) 
        return img_tensor
    except (OSError, UnidentifiedImageError) as e:
        return None

def extract_features(image_path):
    img_tensor = preprocess_for_resnet(image_path)
    if img_tensor is None:
        return None  
    with torch.no_grad():
        features = resnet(img_tensor)
    return features.squeeze().numpy()

## Model Training 

In [25]:
X = []  
y = []  

def train_model(X, y):
    X = np.array(X)
    y = np.array(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = model.score(X_test, y_test)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Model accuracy: {accuracy}")
    print(f"Model F1 score: {f1}")
    
    return model

## Check Image Downloads 

In [6]:
def check_images_already_downloaded(image_links, image_folder):
    print("Checking if images are already downloaded...")
    missing_images = []
    for image_link in tqdm(image_links, desc="Checking images"):
        image_filename = os.path.basename(image_link)
        image_path = os.path.join(image_folder, image_filename)
        if not os.path.exists(image_path):
            missing_images.append(image_link)
    print(f"Check completed. {len(missing_images)} missing images found.")
    return missing_images

## Prediction Function 

In [7]:
def predictor(image_link, category_id, entity_name, model, image_folder):
    image_filename = os.path.basename(image_link)
    image_path = os.path.join(image_folder, image_filename)
    
    print(f"Processing image link: {image_link}")
    print(f"Image will be saved at: {image_path}")

    if not os.path.exists(image_path):
        print(f"Image not found locally. Downloading image: {image_filename}")
        download_images([image_link], image_folder, allow_multiprocessing=False)
        print(f"Image downloaded: {image_filename}")
    
    print(f"Extracting features from image: {image_filename}")
    features = extract_features(image_path)

    if features is None:
        print(f"Skipping prediction due to incomplete or corrupt image: {image_filename}")
        return None

    print(f"Making prediction for entity: {entity_name}")
    prediction = model.predict([features])[0]
    print(f"Prediction completed: {prediction}")
    return prediction

# Main Processing Code

## Training Data Loading

In [11]:
DATASET_FOLDER = './dataset/'
IMAGE_DOWNLOAD_FOLDER_TRAIN = './downloaded_images/'
IMAGE_DOWNLOAD_FOLDER_TEST = './downloaded_images_test/'

print(f"Loading training dataset from folder: {DATASET_FOLDER}")
train_data = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
print("Training dataset loaded.")

missing_train_images = check_images_already_downloaded(train_data['image_link'].tolist(), IMAGE_DOWNLOAD_FOLDER_TRAIN)

if missing_train_images:
    print(f"Found {len(missing_train_images)} images to download for training.")
    download_images(missing_train_images, IMAGE_DOWNLOAD_FOLDER_TRAIN)
    print("Image download process for training completed.")
else:
    print("All training images are already downloaded.")

Loading training dataset from folder: ./dataset/
Training dataset loaded.
Checking if images are already downloaded...


Checking images: 100%|██████████| 263859/263859 [00:32<00:00, 8154.37it/s] 

Check completed. 0 missing images found.
All training images are already downloaded.


## Feature Extraction

In [26]:
print("Starting feature extraction for training images...")

num_images_to_process = 10000
limited_train_data = train_data.head(num_images_to_process)

X_train = []
y_train = []

skipped_images = []
successful_images = []

for index, row in tqdm(limited_train_data.iterrows(), total=limited_train_data.shape[0], desc='Extracting Features'):
    image_link = row['image_link']
    entity_value = row['entity_value']  
    
    image_filename = os.path.basename(image_link)
    image_path = os.path.join(IMAGE_DOWNLOAD_FOLDER_TRAIN, image_filename)
    
    print(f"Extracting features from image: {image_filename}")
    features = extract_features(image_path)
    
    if features is not None:
        X_train.append(features)
        y_train.append(entity_value)
        successful_images.append(image_filename)
    else:
        skipped_images.append(image_filename)
        print(f"Skipping image: {image_filename} due to incomplete or corrupt file.")

print("Feature extraction for training completed.")
print(f"Total images processed: {len(limited_train_data)}")
print(f"Successfully extracted features from {len(successful_images)} images.")
print(f"Skipped {len(skipped_images)} images.")

if skipped_images:
    print("Skipped images:")
    for img in skipped_images:
        print(f" - {img}")

# if X_train:
#     sample_features = np.array(X_train)
#     print("Sample of extracted features:")
#     print(sample_features[:5])  # Print the first 5 feature vectors
#     print("Feature vector shape:", sample_features[0].shape)
# else:
#     print("No features were extracted.")

Starting feature extraction for training images...


Extracting Features:   0%|          | 1/500 [00:00<01:27,  5.68it/s]

Extracting features from image: 61I9XdN6OFL.jpg
Extracting features from image: 71gSRbyXmoL.jpg


Extracting Features:   0%|          | 2/500 [00:00<01:19,  6.24it/s]

Extracting features from image: 61BZ4zrjZXL.jpg


...

Extracting features from image: 719skH5g3DL.jpg
Extracting features from image: 81L5nXqWeLL.jpg


Extracting Features: 100%|█████████▉| 499/500 [01:49<00:00,  5.08it/s]

Extracting features from image: 61m-jQu+ZaL.jpg
Extracting features from image: 71yqw2dcvCL.jpg


Extracting Features: 100%|██████████| 500/500 [01:49<00:00,  4.57it/s]

Feature extraction for training completed.
Total images processed: 500
Successfully extracted features from 500 images.
Skipped 0 images.


## Model Training

In [27]:
print("Training model on extracted features...")
model = train_model(X_train, y_train)
print("Model training completed.")

Training model on extracted features...
Model accuracy: 0.12
Model F1 score: 0.09626984126984127
Model training completed.


## Model Testing

In [28]:
print(f"Loading test dataset from folder: {DATASET_FOLDER}")
test_data = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
print("Test dataset loaded.")

missing_test_images = check_images_already_downloaded(test_data['image_link'].tolist(), IMAGE_DOWNLOAD_FOLDER_TEST)

if missing_test_images:
    print(f"Found {len(missing_test_images)} images to download for testing.")
    download_images(missing_test_images, IMAGE_DOWNLOAD_FOLDER_TEST)  # Downloading test images
    print("Image download process for testing completed.")
else:
    print("All test images are already downloaded.")

print("Starting predictions for test data...")
test_predictions = test_data.copy()
test_predictions['prediction'] = test_predictions.apply(
    lambda row: predictor(row['image_link'], row['group_id'], row['entity_name'], model, IMAGE_DOWNLOAD_FOLDER_TEST), axis=1
)

test_predictions = test_predictions[test_predictions['prediction'].notnull()]
print("Predictions for test data completed.")

y_true = test_predictions['entity_name'] 
y_pred = test_predictions['prediction']

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Model accuracy: {accuracy}")
print(f"Model F1 score: {f1}")

output_filename = os.path.join(DATASET_FOLDER, 'test_out.csv')
print(f"Saving predictions to {output_filename}...")
test_predictions[['index', 'prediction']].to_csv(output_filename, index=False)
print(f"Predictions saved to {output_filename}.")

Loading test dataset from folder: ./dataset/
Test dataset loaded.
Checking if images are already downloaded...


Checking images: 100%|██████████| 88/88 [00:00<00:00, 16886.98it/s]

Check completed. 88 missing images found.
Found 88 images to download for testing.



100%|██████████| 88/88 [00:15<00:00,  5.70it/s]


Image download process for testing completed.
Starting predictions for test data...
Processing image link: https://m.media-amazon.com/images/I/41-NCxNuBxL.jpg
Image will be saved at: ./downloaded_images_test/41-NCxNuBxL.jpg
Extracting features from image: 41-NCxNuBxL.jpg
Making prediction for entity: width
Prediction completed: 72.0 watt
Processing image link: https://m.media-amazon.com/images/I/41-NCxNuBxL.jpg
Image will be saved at: ./downloaded_images_test/41-NCxNuBxL.jpg
Extracting features from image: 41-NCxNuBxL.jpg
Making prediction for entity: depth
Prediction completed: 72.0 watt
Processing image link: https://m.media-amazon.com/images/I/417NJrPEk+L.jpg
Image will be saved at: ./downloaded_images_test/417NJrPEk+L.jpg
Extracting features from image: 417NJrPEk+L.jpg
Making prediction for entity: maximum_weight_recommendation
Prediction completed: 60.0 gram
Processing image link: https://m.media-amazon.com/images/I/417SThj+SrL.jpg
Image will be saved at: ./downloaded_images_test/